In [1]:
%matplotlib inline

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
# TODO: try different values of hyperparameters and check how it will affect the classification performance.

batch_size = 128
learning_rate = 0.0001

In [4]:
# We normalize the data by its mean and variance.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                download=True, transform=transform)

# training validation split 
train_set, val_set = torch.utils.data.random_split(trainset, [50000, 10000])

trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

valloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                     download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [5]:
import torch.nn as nn
import torch.nn.functional as F

# TODO: Implement the LeNet according to the description.
class LeNet(nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(400, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

# We've implemented a multi-layer perceptron model so that you can try to run the training algorithm
# and compare it with LeNet in terms of the classification performance.

class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()
        self.input = nn.Linear(28 * 28, 512)
        self.hidden = nn.Linear(512, 256)
        self.output = nn.Linear(256, 10)
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.sigmoid(self.input(x))
        x = torch.sigmoid(self.hidden(x))
        x = self.output(x)
        return x

net = MLP()

# Uncomment this line after you implement it
net = LeNet()

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

In [7]:
for epoch in range(10):  # loop over the dataset multiple times
    train_loss = 0.0
    train_acc = 0.0
    val_loss = 0.0
    val_acc = 0.0
    test_loss = 0.0
    test_acc = 0.0
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # statistics
        train_loss += loss.item()
        pred = torch.max(outputs, 1)[1]
        train_correct = (pred == labels).sum()
        train_acc += train_correct.item()

    # To get the best learned model, we need to do some statistics.
    # After training, we pick the model with the best validation accuracy.
    with torch.no_grad():
        net.eval()

        for inputs, labels in valloader:

            predicts = net(inputs)

            loss = criterion(predicts, labels)
            val_loss += loss.item()
            pred = torch.max(predicts, 1)[1]
            val_correct = (pred == labels).sum()
            val_acc += val_correct.item()

        for inputs, labels in testloader:

            predicts = net(inputs)
            pred = torch.max(predicts, 1)[1]
            test_correct = (pred == labels).sum()
            test_acc += test_correct.item()

        net.train()
    print("Epoch %d" % epoch )

    print('Training Loss: {:.6f}, Training Acc: {:.6f}, Validation Acc: {:.6f}, Test Acc: {:.6f}'.format(train_loss / (len(train_set)) * 32, train_acc / (len(train_set)), val_acc / (len(val_set)), test_acc / (len(testset))))

print('Finished Training')

Epoch 0
Training Loss: 0.575532, Training Acc: 0.095140, Validation Acc: 0.096100, Test Acc: 0.100500
Epoch 1
Training Loss: 0.573589, Training Acc: 0.095980, Validation Acc: 0.095600, Test Acc: 0.100400
Epoch 2
Training Loss: 0.571227, Training Acc: 0.131200, Validation Acc: 0.187000, Test Acc: 0.192200
Epoch 3
Training Loss: 0.568038, Training Acc: 0.252520, Validation Acc: 0.310300, Test Acc: 0.318800
Epoch 4
Training Loss: 0.562842, Training Acc: 0.381020, Validation Acc: 0.453200, Test Acc: 0.457200
Epoch 5
Training Loss: 0.553123, Training Acc: 0.494660, Validation Acc: 0.527200, Test Acc: 0.532300
Epoch 6
Training Loss: 0.532254, Training Acc: 0.564060, Validation Acc: 0.596400, Test Acc: 0.607500
Epoch 7
Training Loss: 0.477192, Training Acc: 0.645320, Validation Acc: 0.704200, Test Acc: 0.719100
Epoch 8
Training Loss: 0.345636, Training Acc: 0.743960, Validation Acc: 0.776000, Test Acc: 0.785300
Epoch 9
Training Loss: 0.206962, Training Acc: 0.804780, Validation Acc: 0.822200,